In [1]:
"""
MATLAB version AUTHOR:
    Dr. Cynthia Sung (crsung@seas.upenn.edu)
    Modified by Gedaliah Knizhnik (knizhnik@seas.upenn.edu) 08/28/19
Python version transformed AUTHOR:
    Zichen Lao (lao0910@seas.upenn.edu) 06/05/20
"""

import numpy as np

class calculateFK():

    def __init__(self):
        """
        This is the dimension of the Lynx Robot stated as global variable

        """
        # Lynx Dimensions in mm
        self.L1 = 76.2    # distance between joint 0 and joint 1
        self.L2 = 146.05  # distance between joint 1 and joint 2
        self.L3 = 187.325 # distance between joint 2 and joint 3
        self.L4 = 34      # distance between joint 3 and joint 4
        self.L5 = 34      # distance between joint 4 and center of gripper

        # Joint limits
        self.lowerLim = np.array([-1.4, -1.2, -1.8, -1.9, -2.0, -15]).reshape((1, 6))    # Lower joint limits in radians (grip in mm (negative closes more firmly))
        self.upperLim = np.array([1.4, 1.4, 1.7, 1.7, 1.5, 30]).reshape((1, 6))          # Upper joint limits in radians (grip in mm)

    def forward(self, q):
        """
        INPUT:
        q - 1x6 vector of joint inputs [q0,q1,q2,q3,q4,lg]

        OUTPUTS:
        jointPositions - 6 x 3 matrix, where each row represents one
                  joint along the robot. Each row contains the [x,y,z]
                  coordinates of the respective joint's center (mm). For
                  consistency, the first joint should be located at
                  [0,0,0].
        T0e       - a 4 x 4 homogeneous transformation matrix,
                  representing the end effector frame expressed in the
                  base (0) frame
        """
        # Your code starts from here
        # Frame 1 w.r.t Frame 0
        T1 = np.array([[np.cos(q[0]), -np.sin(q[0])*np.cos(-np.pi/2), np.sin(q[0])*np.sin(-np.pi/2), 0],
                       [np.sin(q[0]), np.cos(q[0])*np.cos(-np.pi/2), -np.cos(q[0])*np.sin(-np.pi/2), 0],
                       [0, np.sin(-np.pi/2), np.cos(-np.pi/2), self.L1],
                       [0, 0, 0, 1]])

        # Frame 2 w.r.t Frame 1
        T2 = np.array([[np.cos(q[1]-(np.pi/2)), -np.sin(q[1]-(np.pi/2)), 0, self.L2*np.cos(q[1]-(np.pi/2))],
                       [np.sin(q[1]-(np.pi/2)), np.cos(q[1]-(np.pi/2)), 0, self.L2*np.sin(q[1]-(np.pi/2))],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])

        # Frame 3 w.r.t Frame 2
        T3 = np.array([[np.cos(q[2]+(np.pi/2)), -np.sin(q[2]+(np.pi/2)), 0, self.L3*np.cos(q[2]+(np.pi/2))],
                       [np.sin(q[2]+(np.pi/2)), np.cos(q[2]+(np.pi/2)), 0, self.L3*np.sin(q[2]+(np.pi/2))],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])

        # Frame 4 w.r.t Frame 3
        T4 = np.array([[np.cos(q[3]-(np.pi/2)), -np.sin(q[3]-(np.pi/2))*np.cos(-np.pi/2), np.sin(q[3]-(np.pi/2))*np.sin(-np.pi/2), 0],
                       [np.sin(q[3]-(np.pi/2)), np.cos(q[3]-(np.pi/2))*np.cos(-np.pi/2), -np.cos(q[3]-(np.pi/2))*np.sin(-np.pi/2), 0],
                       [0, np.sin(-np.pi/2), np.cos(-np.pi/2), 0],
                       [0, 0, 0, 1]])
        # Frame 5 w.r.t Frame 4
        T5 = np.array([[np.cos(q[4]), -np.sin(q[4]), 0, 0],
                       [np.sin(q[4]), np.cos(q[4]), 0, 0],
                       [0, 0, 1, self.L4 + self.L5],
                       [0, 0, 0, 1]])

        x = np.empty((6, 4)).reshape((6, 4))
        zeroPos = np.array([0, 0, 0, 1]).reshape((1, 4))
        zeroPos_trans = np.transpose(zeroPos)

        # Position of First Joint (Base Revolute)
        x[0, :] = zeroPos

        # Position of Second Joint (Shoulder Revolute)
        x[1, :] = np.transpose(T1.dot(zeroPos_trans))

        # Position of Third Joint (Elbow Revolute)
        x[2, :] = np.transpose((T1.dot(T2)).dot(zeroPos_trans))

        # Position of Fourth Joint (1st Wrist)
        x[3, :] = np.transpose(((T1.dot(T2)).dot(T3)).dot(zeroPos_trans))

        # Position of Fifth Joint (2nd Wrist)
        x[4, :] = np.transpose((((T1.dot(T2)).dot(T3)).dot(T4)).dot(np.array([0, 0, self.L4, 1]).reshape((4, 1))))

        # Position of Gripper (Base of the Gripper)
        x[5, :] = np.transpose(((((T1.dot(T2)).dot(T3)).dot(T4)).dot(T5)).dot(zeroPos_trans))
        # Outputs the 6x3 of the locations of each joint in the Base Frame
        jointPositions = x[0:6,0:3]

        T0e = ((((T1.dot(T2)).dot(T3)).dot(T4)).dot(T5))
        # Your code ends here

        return jointPositions, T0e

In [2]:
#distPointToBox
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

def distPointToBox(p, box):
    """
    Calculate the distance between a point and an axis-aligned box.
    :param p: Nx3 vector of points [x,y,z]
    :param box: 1x6 vector of minimum and maximum points of box
    :return
    dist - Nx1 vector of distance between the points and the box
            dist > 0 point outside
            dist = 0 point is on or inside box
    unit - Nx3 vector where each row is the corresponding unit vector to the closest spot on the box
        norm(unit) = 1 point is outside the box
        norm(unit)= 0 point is on/inside the box

     Method from MultiRRomero
     @ https://stackoverflow.com/questions/5254838/calculating-distance-between-a-point-and-a-rectangular-box-nearest-point
    """
    # Get box info
    boxMin = np.array([box[0], box[1], box[2]])
    boxMax = np.array([box[3], box[4], box[5]])
    boxCenter = boxMin*0.5 + boxMax*0.5
    p = np.array(p)

    # Get distance info from point to box boundary
    dx = np.amax(np.vstack([boxMin[0] - p[:, 0], p[:, 0] - boxMax[0], np.zeros(p[:, 0].shape)]).T, 1)
    dy = np.amax(np.vstack([boxMin[1] - p[:, 1], p[:, 1] - boxMax[1], np.zeros(p[:, 1].shape)]).T, 1)
    dz = np.amax(np.vstack([boxMin[2] - p[:, 2], p[:, 2] - boxMax[2], np.zeros(p[:, 2].shape)]).T, 1)

    # convert to distance
    distances = np.vstack([dx, dy, dz]).T
    dist = np.linalg.norm(distances, axis=1)

    # Figure out the signs
    signs = np.sign(boxCenter-p)

    # Calculate unit vector and replace with
    unit = distances / dist[:, np.newaxis] * signs
    unit[np.isnan(unit)] = 0
    unit[np.isinf(unit)] = 0
    return dist, unit


# if __name__=='__main__':
#     box = [-1, -1, -1, 1, 1, 1]
#     p = [[0, 0, 0], [1, 1, 1], [2, 2, 2], [3, 3, 3], [-3, -3, -3]]
#     dist, unit = distPointToBox(p, box)
#     print(unit)
#     print(dist)




In [3]:
#Loadmap
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Writen Shane Rozen-Levy 10/3/20
"""

import numpy as np
from collections import namedtuple


def loadmap(filename):
    """
    :param filename: string with the location of the map file
    :return: map struct with boundary and obstacles element
                map.obstacles [Nx6] array of the obstacle boundaries
                map.boundary [6] array of the map boundary
    """
    obstacles = []
    boundary = []
    with open(filename, 'r') as reader:
        # Read file line by line
        line = reader.readline()
        while line != '':  # The EOF char is an empty string
            line = reader.readline()
            # Check to see if first character is b for boundary or block
            if len(line) > 0 and line[0] == 'b':
                words = line.split()
                # Check if block our boundary
                if words[0] == "block":
                    # Append to obstacles array or set to obstacles array
                    if len(obstacles) == 0:
                        obstacles = np.array([[float(words[i]) for i in range(1, len(words))]])
                    else:
                        obstacles = np.append(obstacles, np.array([[float(words[i]) for i in range(1, len(words))]]), axis=0)
                # Set boundary to the last boundary in file
                elif words[0] == "boundary":
                    boundary = np.array([float(words[i])  for i in range(1, len(words))])
    # Make sure atleast one boundary was in the file
    if len(boundary) == 0:
        raise Exception("Map file missing boundary")
    else:
        # Returns the map in a struct
        MyStruct = namedtuple("map", "obstacles boundary")
        return MyStruct(obstacles = obstacles, boundary = boundary)


In [77]:
def calcJacobian(q, joint):
    
    Jacv=np.zeros((3,6)) #zeroes after col: joint+1
    Jacw=np.zeros((3,6)) #zeroes after col: joint+1
    
    #For loop with the size of 3 x joint
    R=calculateFK()
    
    z=[] # np.array(3,joint)
    z.append(np.array([0,0,1]))
    z.append(np.array([-np.sin(q[0]),np.cos(q[0]),0]))
    z.append(np.array([-np.sin(q[0]),np.cos(q[0]),0]))
    z.append(np.array([-np.sin(q[0]),np.cos(q[0]),0]))
    
    Jp,T0e=R.forward(q)
    z.append([T0e[0,2],T0e[1,2],T0e[2,2]])
    z.append([T0e[0,2],T0e[1,2],T0e[2,2]])
    
    for i in range (1,joint+1):
        Jo=Jp[5]-Jp[i-1]
        Jr=np.cross(z[i-1],Jo)
        Jacv[0,i-1]=Jr[0]
        Jacv[1,i-1]=Jr[1]
        Jacv[2,i-1]=Jr[2]

    z = np.array(z)
    z[joint+1:len(z)] *= 0
    Jacw=np.transpose(np.array(z))
    return (np.append(Jacv, Jacw, axis=0))

In [78]:
# import numpy as np

# def calcJacobian(q, joint):
#     """
#     Calculate the Jacobian of a particular joint of the  robot in a given configuration
#     :param q: 1 x 6 vector of joint inputs [q1,q2,q3,q4,q5,q6]
#     :param joint: scalar in [0,6] representing which joint we care about
#     :return: J - 6 x (joint-1) matrix representing the Jacobian
#     """

#     if joint <= 1:
#         return np.array([])
#     FK = calculateFK()
#     jointPositions, T0 = FK.forward(q)

#     Jw1 = np.zeros((3, joint-1))

#     for i in range(joint-1):
#         Jw1[:, i] = T0[range(3), 2, i]

#     Jv2 = np.zeros((3, joint-1))
#     for i in range(joint-1):
#         Jv2[:, i] = np.cross(T0[range(3), 2, i], jointPositions[joint-1, :].T - T0[range(3), 3, i])
#     J = np.vstack((Jv2, Jw1))
#     return J


In [99]:
#potentialFieldPath
import numpy as np
from copy import deepcopy
from time import sleep
import numpy as np
import sys
from random import random as rand
from sys import path
from os import getcwd


def potentialFieldPath(map_struct, qStart, qGoal):
    #need to define these variables
    path=[]
    isDoneArray = []
    qNext,isDone=potentialFieldStep(qStart,map_struct,qGoal)
    path.append(qNext)
    isDoneArray.append(isDone)
    print("1st qCurr:",qNext)
    f=0
    while isDone==False:
        f+=1
        print("iteration #:",f)
        qCurr=path[-1]
        #print("qCurr:",qCurr)
        print("path",path)
        #print("ee path:",path[:][5])
        qNext,isDone=potentialFieldStep(qCurr,map_struct,qGoal)
        path.append(qNext)
        #print("qNext:",qNext)
        isDoneArray.append(isDone)
        
        
#         if(isDoneArray[-1] and isDoneArray[-2]):
#             # robot is stuck
        
    return path

In [116]:
import numpy as np
def potentialFieldStep(qCurr, map_struct, qGoal):

    #constants to control
    zeta=5
    eta=1
    rho0=2
    alpha=1
    
    #intializing vectors
    Fatt=np.zeros((6, 3))
    Frep=np.zeros((6, 3))
    Fnet=np.zeros((6, 3))
    tor=np.zeros(6)
    
    
    R=calculateFK()
    Jp,T0e=R.forward(qCurr)
    Jpg,T0eg=R.forward(qGoal)
    #print("current position:",Jp[3])
    #print("goal position:",Jpg[3])
    print("distance away:",Jp-Jpg)

    rho = []
    unit=[]
    for box in map_struct.obstacles:
        dist_i, unit_i = distPointToBox(Jp, box)
        rho.append(dist_i)
        unit.append(unit_i)

    
    for i in range(0,6):
        # Compute force of repulsion
        for j in range(len(rho)):
            # distance between joint and current obstacle
            drho=unit_i[j]
            # Add current obstacle's repulsive force to accumulated repulsive force on joint
            Frep[i] += eta*(1/rho[j][i] - 1/rho0)*(1/rho[j][i]**2)*drho
            
        Fatt[i] = -zeta*(Jp[i] - Jpg[i])
        Fnet[i] = Fatt[i] + Frep[i]
        Jac=calcJacobian(qCurr,i)[:3,i-1]
        print("Jacobian:",Jac)
        tor[i] = np.matmul(Jac.T,Fnet[i])
    #print("force of repulsion ", Frep)
    print("force of attraction ", Fatt)
    #print("Net force:", Fnet)
    #print("torque:",tor)
    print(tor.shape)
    qNext = qCurr+alpha*((tor)/np.linalg.norm(tor))
    
    
    #Updating isDone
    done=0
    for r in range(0,6):
        if abs(qNext[r]-qCurr[r])<0.05:
            done=+1

    if done==5:
        isDone=True
    else:
        isDone=False
    
    return qNext,isDone



In [117]:
qStart=[0,0,0,0,0,0]
qGoal=[0,0,-np.pi/2,0,0,0]
map_struct = loadmap("Lab5/maps/emptyMap.txt")
print(map_struct.obstacles)
potentialFieldPath(map_struct,qStart,qGoal)

[]
distance away: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.87325000e+02  1.14703481e-14 -1.87325000e+02]
 [ 2.21325000e+02  1.35522476e-14 -2.21325000e+02]
 [ 2.55325000e+02  1.56341472e-14 -2.55325000e+02]]
Jacobian: [0. 0. 0.]
Jacobian: [ 4.77918413e-15  2.55325000e+02 -0.00000000e+00]
Jacobian: [ 146.05     0.    -255.325]
Jacobian: [   0.       0.    -255.325]
Jacobian: [  0.   0. -68.]
Jacobian: [ 1.27479582e-31 -2.08189956e-15 -3.94430453e-31]
force of attraction  [[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.36625000e+02 -5.73517404e-14  9.36625000e+02]
 [-1.10662500e+03 -6.77612382e-14  1.10662500e+03]
 [-1.27662500e+03 -7.81707360e-14  1.27662500e+03]]
(6,)
1st qCurr: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.53889805e-01
 -3.00157024e-01 -2.

distance away: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.87325000e+02  1.14703481e-14 -1.87325000e+02]
 [ 1.56779205e+02  1.26379473e-14 -2.06393342e+02]
 [ 1.26233410e+02  1.38055465e-14 -2.25461685e+02]]
Jacobian: [0. 0. 0.]
Jacobian: [ 6.6077848e-15  1.2623341e+02 -0.0000000e+00]
Jacobian: [ 175.91331517    0.         -126.23340968]
Jacobian: [  29.86331517    0.         -126.23340968]
Jacobian: [29.86331517  0.         61.09159032]
Jacobian: [-2.95822839e-31 -5.32907052e-15 -1.97215226e-31]
force of attraction  [[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.36625000e+02 -5.73517404e-14  9.36625000e+02]
 [-7.83896024e+02 -6.31897365e-14  1.03196671e+03]
 [-6.31167048e+02 -6.90277327e-14  1.12730842e+03]]
(6,)
iteration #: 25
path [array([ 0.00000000e+00,  0.00

        6.24391340e+00, -5.14148889e-32])]
distance away: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.87325000e+02  1.14703481e-14 -1.87325000e+02]
 [ 1.94853572e+02  1.55824673e-14 -2.54481004e+02]
 [ 2.02382144e+02  1.96945866e-14 -3.21637008e+02]]
Jacobian: [0. 0. 0.]
Jacobian: [ 7.18744716e-16  2.02382144e+02 -0.00000000e+00]
Jacobian: [  79.73799199    0.         -202.38214427]
Jacobian: [ -66.31200801    0.         -202.38214427]
Jacobian: [-66.31200801   0.         -15.05714427]
Jacobian: [7.88860905e-31 4.44089210e-15 7.88860905e-31]
force of attraction  [[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.36625000e+02 -5.73517404e-14  9.36625000e+02]
 [-9.74267861e+02 -7.79123367e-14  1.27240502e+03]
 [-1.01191072e+03 -9.84729331e-14  1.60818504e+03]]
(6,)
itera

        8.76762871e+00, -4.10990444e-32])]
distance away: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.87325000e+02  1.14703481e-14 -1.87325000e+02]
 [ 2.16043745e+02  1.46666656e-14 -2.39524827e+02]
 [ 2.44762489e+02  1.78629832e-14 -2.91724654e+02]]
Jacobian: [0. 0. 0.]
Jacobian: [ 2.55034813e-15  2.44762489e+02 -0.00000000e+00]
Jacobian: [ 109.65034569    0.         -244.7624892 ]
Jacobian: [ -36.39965431    0.         -244.7624892 ]
Jacobian: [-36.39965431   0.         -57.4374892 ]
Jacobian: [-1.97215226e-31  1.77635684e-15 -3.94430453e-31]
force of attraction  [[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.36625000e+02 -5.73517404e-14  9.36625000e+02]
 [-1.08021872e+03 -7.33333282e-14  1.19762414e+03]
 [-1.22381245e+03 -8.93149160e-14  1.45862327e+03]]
(6,)
it

        9.68728109e+00, -4.71020161e-32])]
distance away: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.87325000e+02  1.14703481e-14 -1.87325000e+02]
 [ 1.65994882e+02  1.51734900e-14 -2.47801897e+02]
 [ 1.44664763e+02  1.88766319e-14 -3.08278793e+02]]
Jacobian: [0. 0. 0.]
Jacobian: [ 1.53669947e-15  1.44664763e+02 -0.00000000e+00]
Jacobian: [  93.0962068     0.         -144.66476341]
Jacobian: [ -52.9537932     0.         -144.66476341]
Jacobian: [-52.9537932   -0.          42.66023659]
Jacobian: [ 0.00000000e+00 -1.06581410e-14 -1.57772181e-30]
force of attraction  [[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.36625000e+02 -5.73517404e-14  9.36625000e+02]
 [-8.29974409e+02 -7.58674499e-14  1.23900948e+03]
 [-7.23323817e+02 -9.43831593e-14  1.54139397e+03]]
(6,)
it

...

path [array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -9.53889805e-01,
       -3.00157024e-01, -2.00850494e-33]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.89437483e+00,
       -6.39992165e-01, -3.12422903e-33]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -2.88037495e+00,
       -8.06736777e-01, -1.96694851e-33]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -3.79855972e+00,
       -4.10584382e-01, -1.96230894e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -4.45684663e+00,
        3.42182742e-01, -2.85780630e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -5.22816399e+00,
        9.78633479e-01, -2.88855152e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -6.22095099e+00,
        1.09852497e+00, -2.95718165e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -7.17807303e+00,
        8.08839986e-01, -3.17375421e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.000000

        1.94315901e+01, -7.36776352e-32])]
distance away: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.87325000e+02  1.14703481e-14 -1.87325000e+02]
 [ 2.12882771e+02  1.21792220e-14 -1.98901790e+02]
 [ 2.38440542e+02  1.28880960e-14 -2.10478580e+02]]
Jacobian: [0. 0. 0.]
Jacobian: [ 7.52523538e-15  2.38440542e+02 -0.00000000e+00]
Jacobian: [ 190.8964202     0.         -238.44054162]
Jacobian: [  44.8464202     0.         -238.44054162]
Jacobian: [ 44.8464202    0.         -51.11554162]
Jacobian: [ 1.97215226e-31  3.55271368e-15 -3.94430453e-31]
force of attraction  [[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.36625000e+02 -5.73517404e-14  9.36625000e+02]
 [-1.06441385e+03 -6.08961101e-14  9.94508950e+02]
 [-1.19220271e+03 -6.44404798e-14  1.05239290e+03]]
(6,)
it

path [array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -9.53889805e-01,
       -3.00157024e-01, -2.00850494e-33]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.89437483e+00,
       -6.39992165e-01, -3.12422903e-33]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -2.88037495e+00,
       -8.06736777e-01, -1.96694851e-33]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -3.79855972e+00,
       -4.10584382e-01, -1.96230894e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -4.45684663e+00,
        3.42182742e-01, -2.85780630e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -5.22816399e+00,
        9.78633479e-01, -2.88855152e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -6.22095099e+00,
        1.09852497e+00, -2.95718165e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -7.17807303e+00,
        8.08839986e-01, -3.17375421e-32]), array([ 0.00000000e+00,  0.00000000e+00,  0.000000

distance away: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.87325000e+02  1.14703481e-14 -1.87325000e+02]
 [ 1.89016982e+02  1.14729276e-14 -1.87367126e+02]
 [ 1.90708964e+02  1.14755070e-14 -1.87409252e+02]]
Jacobian: [0. 0. 0.]
Jacobian: [ 8.93782429e-15  1.90708964e+02 -0.00000000e+00]
Jacobian: [ 213.96574774    0.         -190.70896356]
Jacobian: [  67.91574774    0.         -190.70896356]
Jacobian: [67.91574774  0.         -3.38396356]
Jacobian: [ 2.22637502e-31 -1.97619698e-14 -9.58150147e-33]
force of attraction  [[-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.36625000e+02 -5.73517404e-14  9.36625000e+02]
 [-9.45084909e+02 -5.73646378e-14  9.36835631e+02]
 [-9.53544818e+02 -5.73775352e-14  9.37046261e+02]]
(6,)
iteration #: 144
path [array([ 0.00000000e+00,  

KeyboardInterrupt: 